In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

## Building Recommender Systems Easily with Merlin Models

In this example notebook, we use the [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset to build our recommender system models. We define different deep learning-based model architectures, and then train and evaluate the models only in couple lines of code.

### Learning objectives
- Preparing the data with NVTabular
- Training different recommender system models with Merlin Models

## Feature Engineering with NVTabular

When we work on a new recommender systems, we explore the dataset, first.

In [2]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import cudf
import glob
import gc

import nvtabular as nvt
from nvtabular.ops import *
from example_utils import workflow_fit_transform

from merlin.schema.tags import Tags
from merlin.schema import Schema

import merlin.models.tf as mm
import merlin.models.tf.dataset as tf_dataloader

from merlin.io.dataset import Dataset
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata
from merlin.models.tf.blocks.core.aggregation import CosineSimilarity

import tensorflow as tf

2022-03-22 21:44:21.999477: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 21:44:23.143720: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-03-22 21:44:23.143859: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


First, we define our input and output paths.

In [3]:
train_path = '/workspace/data/train/*.parquet'
test_path = '/workspace/data/test/*.parquet'
output_path = '/workspace/ranking/processed/'

<a id="etl"></a>
ETL Workflow:

In [4]:
%%time

user_id = ["user_id"] >> AddMetadata(tags=[Tags.USER_ID, Tags.USER]) >> Categorify()
item_id = ["item_id"] >> AddMetadata(tags=[Tags.ITEM_ID, Tags.ITEM]) >> Categorify()

item_features = ["item_category", "item_shop", "item_brand"] >> AddMetadata(tags=[Tags.ITEM]) >> nvt.ops.Categorify()

user_features = ['user_shops', 'user_profile', 'user_group', 
       'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
       'user_geography', 'user_intentions', 'user_brands', 'user_categories'] \
    >> AddMetadata(tags=[Tags.USER]) >> nvt.ops.Categorify()

targets = ["click"] >> AddMetadata(tags=[str(Tags.BINARY_CLASSIFICATION), "target"])

outputs = user_id+item_id+item_features+user_features+targets

workflow_fit_transform(outputs, train_path, test_path, output_path)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


CPU times: user 17.7 s, sys: 21.8 s, total: 39.6 s
Wall time: 44.4 s


We will also use a util function to wrap up the workflow transform to a one line of code.

## Building Recommender Systems

NVTabular exported the schema file of our processed dataset. Merlin Models library relies on a schema object that takes the input features as input and automatically builds all necessary layers to represent, normalize and aggregate input features. `schema.pbtxt` is a protobuf text file contains features metadata, including statistics about features such as cardinality, min and max values and also tags based on their characteristics and dtypes (e.g., categorical, continuous, list, item_id). The metadata information loaded from Schema and their tags are used to automatically set the parameters of Merlin models.

We use the `schema` object to define our model.

In [5]:
schema = TensorflowMetadata.from_proto_text_file(output_path + '/train/').to_merlin_schema()

In [6]:
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]

In [7]:
schema.column_names

['user_id',
 'item_id',
 'item_category',
 'item_shop',
 'item_brand',
 'user_shops',
 'user_profile',
 'user_group',
 'user_gender',
 'user_age',
 'user_consumption_2',
 'user_is_occupied',
 'user_geography',
 'user_intentions',
 'user_brands',
 'user_categories',
 'click']

### Initialize Dataloaders

We're ready to start training, for that, we need to initialize the dataloaders. We'll use Merlin `BatchedDataset` class for reading chunks of parquet files. `BatchedDataset` asynchronously iterate through CSV or Parquet dataframes on GPU by leveraging an NVTabular `Dataset`. To read more about Merlin optimized dataloaders visit [here](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/dataset.py#L141).

In [8]:
batch_size = 16*1024
train = Dataset(os.path.join(output_path + '/train/*.parquet'), part_size="500MB")
valid = Dataset(os.path.join(output_path + '/test/*.parquet'), part_size="500MB")

### DLRM Model

Deep Learning Recommendation Model [(DLRM)](https://arxiv.org/abs/1906.00091) architecture is a popular neural network model originally proposed by Facebook in 2019. The model was introduced as a personalization deep learning model that uses embeddings to process sparse features that represent categorical data and a multilayer perceptron (MLP) to process dense features, then interacts these features explicitly using the statistical techniques proposed in [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5694074).

![DLRM](./images/DLRM.png)


DLRM accepts two types of features: categorical and numerical. 
- For each categorical feature, an embedding table is used to provide dense representation to each unique value. 
- For numerical features, they are fed to model as dense features, and then transformed by a simple neural network referred to as "bottom MLP". This part of the network consists of a series of linear layers with ReLU activations. 
- The output of the bottom MLP and the embedding vectors are then fed into the dot product interaction operation (see Pairwise interaction step). The output of "dot interaction" is then concatenated with the features resulting from the bottom MLP (we apply a skip-connection there) and fed into the "top MLP" which is also a series of dense layers with activations ((a fully connected NN). 
- The model outputs a single number (here we use sigmoid function to generate probabilities) which can be interpreted as a likelihood of a certain user clicking on an ad, watching a movie, or viewing a news page.


Steps:<br>
* Change the model to `DLRMModel`
* Rerun the pipeline from there from model.fit

In [9]:
model = mm.DLRMModel(
    schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column, metrics=[tf.keras.metrics.AUC()])
)

In [12]:
%%time
model.compile('adam', run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=batch_size, epochs=1)

2442/2442 [==============================] - ETA: 0s - auc: 0.6501 - loss: 0.1600 - regularization_loss: 0.0000e+00 - total_loss: 0.1600

2022-03-22 21:52:27.852502: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/then/_0/cond/cond/branch_executed/_161


2442/2442 [==============================] - 296s 117ms/step - auc: 0.6501 - loss: 0.1599 - regularization_loss: 0.0000e+00 - total_loss: 0.1599 - val_auc: 0.6048 - val_loss: 0.1207 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.1207
CPU times: user 7min 54s, sys: 1min 32s, total: 9min 27s
Wall time: 4min 57s


Save the model

In [11]:
model.save('dlrm')

INFO:tensorflow:Assets written to: dlrm/assets


INFO:tensorflow:Assets written to: dlrm/assets
